# Excitation function plotting script
## Script for plotting the excitation functions from HIVAP and corresponding experimental data

### Create a reaction object:
~~~
Reaction(
        projectile,
        target,
        cn,
        barfac,
        sigr,
        unit="mb",
        evap_channel="xn",
        bf_diff=0.00,
        plot_diff=True,
        channels_to_plot=[],
        lowYRange=0,
        highYRange=0,
        lowXRange=0,
        highXRange=0,
        exp_data: ExpData = None,
        plot_exp_data=True
)
~~~

unit - change units in which data are plotted: 'mb' (default), 'ub' or 'nb'

evap_channel - change which evap. channels are displayed: 'xn' (defaul), 'pxn' or '2pxn'

bf_diff - specify a BARFAC "uncertainty", which will be displayed as a shaded area around the main value: 0.0 (default)

plot_diff - turn off the BARFAC difference plotting: True default

channels_to_plot - specify which channels are to be plotted

low[axis]Range/high[axis]Range - change the range of [axis] manually

### Experimental data

Experimental data are entered in the following form:

~~~
ExpData(
    E_lab = [laboratory energies],
    cs_data = {
        'Isotope': 
        [
            Cross section values
        ]
    },
    error_data = {
        'Isotope':
        [
            [low assymetric error or symmetric error],
            [high assymetric error; if symmetric error is used, don't fill this array]
        ]
    }
)
~~~

E_lab - energies in the laboratory frame

Isotope - must be also included in the Reaction's "channels_to_plot" or in HIVAP data

Cross section values - array of cross sections in milibarns(!) corresponding to the E_lab - must be of the same size

error_data - optional, in milibarns(!);

'Isotope' in error_data must be also in cs_data

Both arrays (or the first one if symmetric errors are used) for each isotope in error_data must have the same size as E_lab

---

#### Assymetric error example:
~~~
ExpData(E_lab=[230, 240], {'193At': [0.15, 15]}, error_data={'193At': [[0.1, 10], [0.2, 20]]})
~~~
Two data points:
- at 230 MeV - 0.15 mb with (-0.1 +0.2) assymetric errors
- at 240 MeV - 15 mb with (-10 + 20) assymetric errors

---

#### Symmetric error example:
~~~
ExpData(E_lab=[230, 240], {'193At': [0.15, 15]}, error_data={'193At': [[0.1, 10]]})
~~~

Two data points:
- at 230 MeV - 0.15 mb +- 0.1 mb
- at 240 MeV - 15 mb +- 10 mb

In [62]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from math import log10, floor, ceil
import re
from typing import List, Dict
import json
import os

import warnings
import time

warnings.filterwarnings("ignore")

# turn off interactive mode
plt.ioff()

In [63]:
def splitIsotope(isotope):
    parts = re.findall(r"(\d+|[a-zA-Z]+)", isotope)
    return parts

PATH_TO_HIVAPS = "/home/adam/skola/phd/hivap/HIVAP-CODE/Hivap/MyHivap/results"

colors = [
    "black",
    "red",
    "mediumseagreen",
    "blue",
    "goldenrod",
    "fuchsia",
    "turquoise",
    "darkorange",
    "blueviolet",
    "darkgrey",
    "firebrick",
    "darkkhaki",
]

markers = ['o', 'v', '^', 's', '*', 'D', 'X', 'p', 'P', 'h', '<', '>']

f = open('periodicTable.json')
periodicTable = json.load(f)

plt.rcParams.update({"font.size": 18})

label_fontsize = 20
labelpad = 10

In [64]:
class ExpData:
    def __init__(self, E_lab: List[float], cs_data: Dict[str, List[float]], error_data: Dict[str, List[List[float]]] = {}):
        self.E_lab = E_lab
        
        self.__checkIsotopesInData(cs_data, error_data)
        self.__checkCSDataLength(cs_data)
        self.__checkErrorDataLength(error_data)
        self.cs_data = cs_data
        self.error_data = error_data
        
    def __checkCSDataLength(self, data):
        # check that there are CS data for all listed E_lab energies
        for isotope in list(data.keys()):
            if len(self.E_lab) != len(data[isotope]):
                raise ValueError(
                    f"Non-matching length between E_lab and CS data in {isotope}"
                )
    
    def __checkErrorDataLength(self, data):
        # check that there are error data for all listed E_lab energies and if there are assymetric errors, check their dimension as well
        for isotope in list(data.keys()):
            if data[isotope]:
                if len(self.E_lab) != len(data[isotope][0]):
                    raise ValueError(
                        f"Non-matching length between E_lab and low-error data in {isotope}"
                    )
                    
                if len(data[isotope]) == 2 and len(data[isotope][0]) != len(data[isotope][1]):
                    raise ValueError(
                        f"Non-matching length between high- and low-error data in {isotope}"
                    )

    def __checkIsotopesInData(self, cs_data, error_data):
        # check that isotopes that have errors are also in the CS data
        evap_residues = set(cs_data.keys())
        error_data_isotopes = set(error_data.keys())
        if not error_data_isotopes.issubset(evap_residues):
            raise ValueError('Non-matching isotopes in error data and in CS data')

    def getEvapChannelExpData(self, evap_channel):
        exp_data = []
        if evap_channel not in self.cs_data.keys():
            raise ValueError(
                f"Evaporation channel {evap_channel} is not in the entered exp. data"
            )
        
        for index, _ in enumerate(self.E_lab):
            if evap_channel not in self.error_data.keys():
            # if not self.error_data[evap_channel]:                
                exp_data.append([self.E_lab[index], self.cs_data[evap_channel][index], np.nan, np.nan])
            elif len(self.error_data[evap_channel]) == 1:
                exp_data.append([self.E_lab[index], self.cs_data[evap_channel][index], self.error_data[evap_channel][0][index], self.error_data[evap_channel][0][index]])
            elif len(self.error_data[evap_channel]) == 2:
                exp_data.append([self.E_lab[index], self.cs_data[evap_channel][index], self.error_data[evap_channel][0][index], self.error_data[evap_channel][1][index]])
                                
        return exp_data
    
    def getAllExpData(self):
        exp_data = {}
        
        for isotope in self.cs_data:
            exp_data[isotope] = self.getEvapChannelExpData(isotope)
            
        return exp_data
    
    def getMaxChannelCS(self, isotope):
        return max(self.cs_data[isotope])
    
    def getExpEvapResidues(self):
        return list(self.cs_data.keys())

    def getMinCS(self):
        minimum = float('inf')

        for isotope in self.cs_data.keys():
            if np.nanmin(self.cs_data[isotope]) < minimum:
                minimum = np.nanmin(self.cs_data[isotope])
        return minimum
    
    def getMaxCS(self):
        maximum = 0

        for isotope in self.cs_data.keys():
            if maximum < np.nanmax(self.cs_data[isotope]):
                maximum = np.nanmax(self.cs_data[isotope])
        return maximum
    
    def getMaxE_lab(self):
        return max(self.E_lab)
    
    def getMinE_lab(self):
        return min(self.E_lab)

    def scaleExpData(self, unit):
        if unit not in ['mb', 'ub', 'nb', 'pb']:
            raise ValueError('Invalid cross section unit')
        scale = 1
        
        if unit == "mb":
            scale = 1
        elif unit == "ub":
            scale = 1e3
        elif unit == "nb":
            scale = 1e6
        elif unit == "pb":
            scale = 1e9

        for isotope in self.cs_data.keys():
            self.cs_data[isotope] = [scale*i for i in self.cs_data[isotope]]
            try:
                self.error_data[isotope][0] = [scale * i for i in self.error_data[isotope][0]]
                self.error_data[isotope][1] = [scale * i for i in self.error_data[isotope][1]]
            except:
                pass



In [193]:
class Reaction:
    def __init__(
        self,
        projectile,
        target,
        cn,
        barfac,
        sigr,
        unit="mb",
        evap_channel="xn",
        bf_diff=0.00,
        plot_diff=True,
        channels_to_plot=None,
        barrier_model="cps",
        lowYRange=0,
        highYRange=0,
        lowXRange=0,
        highXRange=0,
        exp_data: ExpData = None,
        plot_exp_data=True,
        plot_maxCS_data=False,
        show_bass_barrier=False,
        reaction_info_note="",
        save_note="",
    ):
        self.projectile = projectile
        self.target = target
        self.cn = cn
        self.barfac = barfac
        if unit not in ["mb", "ub", "nb", "pb"]:
            raise ValueError("Invalid cross section unit")
        if barrier_model == "sierk":
            self.barrier_model = barrier_model
            print("Using Sierk calculations")
            
        else:
            barrier_model = ""
            self.barrier_model = barrier_model
            print("Using CPS calculations")
        
        self.unit = unit
        self.evap_channel = evap_channel
        self.sigr = sigr
        self.bf_diff = bf_diff
        self.plot_diff = plot_diff
        self.data_ifus0, self.data_ifus10 = None, None
        (
            self.data_ifus0_bf_low,
            self.data_ifus0_bf_high,
            self.data_ifus10_bf_low,
            self.data_ifus10_bf_high,
        ) = (None, None, None, None)
        self.channels_to_plot = channels_to_plot
        self.lowYRange = lowYRange
        self.highYRange = highYRange
        self.lowXRange = lowXRange
        self.highXRange = highXRange
        if exp_data:
            self.exp_data = exp_data
            self.__processExpData()
        self.__getData()
        self.plot_exp_data = plot_exp_data
        self.plot_maxCS_data = plot_maxCS_data
        self.show_bass_barrier = show_bass_barrier
        self.reaction_info_note = reaction_info_note
        self.save_note = save_note

        if channels_to_plot == None:
            self.channels_to_plot = self.data_ifus0["isotope"].unique()
        else:
            # add to self.channels_to_plot only those that are in the data
            self.channels_to_plot = []
            for channel in channels_to_plot:
                if np.any(self.data_ifus0["isotope"].unique() == channel):
                    self.channels_to_plot.append(channel)

        if lowYRange == 0:
            self.lowYRange = self.__lowYRange()

        if highYRange == 0:
            self.highYRange = self.__highYRange()

        if lowXRange == 0:
            self.lowXRange = self.__lowXRange()

        if highXRange == 0:
            self.highXRange = self.__highXRange()

        if self.bf_diff > 0:
            self.__getBfDiffData()

    def isotopeA(self, isotope):
        return int(splitIsotope(isotope)[0])

    def isotopeElement(self, isotope):
        return splitIsotope(isotope)[1]

    def isotopeZ(self, isotope):
        for element in periodicTable["elements"]:
            if element["symbol"] == self.isotopeElement(isotope):
                return element["number"]
        return None

    def getAllIsotopes(self):
        # first check if IFUS0 and IFUS10 have the same isotopes, then return all of them in a list
        if (
            self.data_ifus0["isotope"].unique() == self.data_ifus10["isotope"].unique()
        ).all():
            return self.data_ifus0["isotope"].unique()
        else:
            raise ValueError("IFUS0 and IFUS10 do not contain the same isotopes")
            
    def getReactionTitle(self):
        return self.projectile + "+" + self.target + "->" + self.cn
        

    def __getData(self):
        # get data into variable
        try:
            self.data_ifus0 = pd.read_csv(
                PATH_TO_HIVAPS 
                + str("" if len(self.barrier_model) == 0 else "_" + self.barrier_model)
                + "/"
                + self.cn.lower()
                + "/data/"
                + self.projectile
                + "_"
                + self.target
                + "_"
                + self.cn
                + "_"
                + self.evap_channel
                + "_"
                + "IFUS0"
                + "_barfac"
                + f"{self.barfac:.2f}"
                + "_sigr"
                + str(self.sigr)
                + ".dat",
                sep="\t",
            )

            self.data_ifus10 = pd.read_csv(
                PATH_TO_HIVAPS
                + str("" if len(self.barrier_model) == 0 else "_" + self.barrier_model)
                + "/"
                + self.cn.lower()
                + "/data/"
                + self.projectile
                + "_"
                + self.target
                + "_"
                + self.cn
                + "_"
                + self.evap_channel
                + "_"
                + "IFUS10"
                + "_barfac"
                + f"{self.barfac:.2f}"
                + "_sigr"
                + str(self.sigr)
                + ".dat",
                sep="\t",
            )

            self.data_ifus0 = self.data_ifus0.melt(
                id_vars=["E_lab", "E*/MeV"], var_name="isotope", value_name="CS"
            )
            self.data_ifus10 = self.data_ifus10.melt(
                id_vars=["E_lab", "E*/MeV"], var_name="isotope", value_name="CS"
            )

            self.__scaleData(self.data_ifus0)
            self.__scaleData(self.data_ifus10)

        except:
            print(
                f"No data found for the reaction {self.projectile} + {self.target} -> {self.cn}*"
            )
            self.data_ifus0, self.data_ifus10 = None, None

    def __getBfDiffData(self):
        # get data into variable
        try:
            self.data_ifus0_bf_low = pd.read_csv(
                PATH_TO_HIVAPS
                + str("" if len(self.barrier_model) == 0 else "_" + self.barrier_model)
                + "/"
                + self.cn.lower()
                + "/data/"
                + self.projectile
                + "_"
                + self.target
                + "_"
                + self.cn
                + "_"
                + self.evap_channel
                + "_"
                + "IFUS0"
                + "_barfac"
                + f"{self.barfac-self.bf_diff:.2f}"
                + "_sigr"
                + str(self.sigr)
                + ".dat",
                sep="\t",
            )

            self.data_ifus0_bf_high = pd.read_csv(
                PATH_TO_HIVAPS
                + str("" if len(self.barrier_model) == 0 else "_" + self.barrier_model)
                + "/"
                + self.cn.lower()
                + "/data/"
                + self.projectile
                + "_"
                + self.target
                + "_"
                + self.cn
                + "_"
                + self.evap_channel
                + "_"
                + "IFUS0"
                + "_barfac"
                + f"{self.barfac+self.bf_diff:.2f}"
                + "_sigr"
                + str(self.sigr)
                + ".dat",
                sep="\t",
            )

            self.data_ifus10_bf_low = pd.read_csv(
                PATH_TO_HIVAPS
                + str("" if len(self.barrier_model) == 0 else "_" + self.barrier_model)
                + "/"
                + self.cn.lower()
                + "/data/"
                + self.projectile
                + "_"
                + self.target
                + "_"
                + self.cn
                + "_"
                + self.evap_channel
                + "_"
                + "IFUS10"
                + "_barfac"
                + f"{self.barfac-self.bf_diff:.2f}"
                + "_sigr"
                + str(self.sigr)
                + ".dat",
                sep="\t",
            )

            self.data_ifus10_bf_high = pd.read_csv(
                PATH_TO_HIVAPS
                + str("" if len(self.barrier_model) == 0 else "_" + self.barrier_model)
                + "/"
                + self.cn.lower()
                + "/data/"
                + self.projectile
                + "_"
                + self.target
                + "_"
                + self.cn
                + "_"
                + self.evap_channel
                + "_"
                + "IFUS10"
                + "_barfac"
                + f"{self.barfac+self.bf_diff:.2f}"
                + "_sigr"
                + str(self.sigr)
                + ".dat",
                sep="\t",
            )

            self.data_ifus0_bf_low = self.data_ifus0_bf_low.melt(
                id_vars=["E_lab", "E*/MeV"], var_name="isotope", value_name="CS"
            )
            self.data_ifus0_bf_high = self.data_ifus0_bf_high.melt(
                id_vars=["E_lab", "E*/MeV"], var_name="isotope", value_name="CS"
            )

            self.data_ifus10_bf_low = self.data_ifus10_bf_low.melt(
                id_vars=["E_lab", "E*/MeV"], var_name="isotope", value_name="CS"
            )
            self.data_ifus10_bf_high = self.data_ifus10_bf_high.melt(
                id_vars=["E_lab", "E*/MeV"], var_name="isotope", value_name="CS"
            )

            self.__scaleData(self.data_ifus0_bf_low)
            self.__scaleData(self.data_ifus0_bf_high)
            self.__scaleData(self.data_ifus10_bf_low)
            self.__scaleData(self.data_ifus10_bf_high)

        except:
            print(
                f"No data found for difference plotting for reaction {self.projectile} + {self.target} -> {self.cn}*"
            )
            (
                self.data_ifus0_bf_low,
                self.data_ifus0_bf_high,
                self.data_ifus10_bf_low,
                self.data_ifus10_bf_high,
            ) = (None, None, None, None)

    def __scaleData(self, dataframe):
        scale = 1
        if self.unit == "mb":
            scale = 1
        elif self.unit == "ub":
            scale = 1e3
        elif self.unit == "nb":
            scale = 1e6
        elif self.unit == "pb":
            scale = 1e9

        dataframe["CS"] = dataframe["CS"].multiply(scale)
        
    def roundMinYRange(self, value):
        return 10 ** round(log10(value)) / 10

    def roundMaxYRange(self, value):
        return 10 ** round(log10(value)) * 10

    def __lowYRange(self):
        # find maximum of the 'smallest' excit. function (IFUS10) to be plotted and then set the minimum to 1 order lower, or min of exp data if it is lower
        lowest_max = float("inf")
        for channel in self.channels_to_plot:
            if (
                self.data_ifus10[self.data_ifus10["isotope"] == channel]["CS"].max() > 0
                and self.data_ifus10[self.data_ifus10["isotope"] == channel]["CS"].max()
                < lowest_max
            ):
                lowest_max = self.data_ifus10[self.data_ifus10["isotope"] == channel][
                    "CS"
                ].max()

        try:
            return self.roundMinYRange(
                self.exp_data.getMinCS()
                if self.exp_data.getMinCS() < lowest_max
                else lowest_max
            )
        except:
            return self.roundMinYRange(lowest_max)

    def __highYRange(self):
        # find maximum of the 'largest' excit. function (IFUS10) to be plotted and set the maximum to 1 order higher, or max of exp data if it is higher
        highest_max = 0
        for channel in self.channels_to_plot:
            if (
                self.data_ifus10[self.data_ifus10["isotope"] == channel]["CS"].max() > 0
                and self.data_ifus10[self.data_ifus10["isotope"] == channel]["CS"].max()
                > highest_max
            ):
                highest_max = self.data_ifus10[self.data_ifus10["isotope"] == channel][
                    "CS"
                ].max()

        try:
            return self.roundMaxYRange(
                self.exp_data.getMaxCS()
                if self.exp_data.getMaxCS() > highest_max
                else highest_max
            )
        except:
            return self.roundMaxYRange(highest_max)

    def roundMinXRange(self, value):
        return floor(value / 10) * 10

    def roundMaxXRange(self, value):
        return ceil(value / 10) * 10

    def __lowXRange(self):
        # get minimum energy for which the CS value is larger than lowYRange or minimum of  E_lab of exp data if it is lower
        try:
            return min(
                self.roundMinXRange(
                    self.data_ifus10[
                        self.data_ifus10["isotope"].isin(self.channels_to_plot)
                    ][self.data_ifus10["CS"] > self.lowYRange]["E_lab"].min()
                ),
                self.exp_data.getMinE_lab() - 5,
            )
        except:
            return self.roundMinXRange(
                self.data_ifus10[
                    self.data_ifus10["isotope"].isin(self.channels_to_plot)
                ][self.data_ifus10["CS"] > self.lowYRange]["E_lab"].min()
            )

    def __highXRange(self):
        # get maximum energy for which the CS value is larger than lowYRange or minimum of  E_lab of exp data if it is lower
        try:
            return max(
                self.roundMaxXRange(
                    self.data_ifus10[
                        self.data_ifus10["isotope"].isin(self.channels_to_plot)
                    ][self.data_ifus10["CS"] > self.lowYRange]["E_lab"].max()
                ),
                self.exp_data.getMaxE_lab() + 5,
            )
        except:
            return self.roundMaxXRange(
                self.data_ifus10[
                    self.data_ifus10["isotope"].isin(self.channels_to_plot)
                ][self.data_ifus10["CS"] > self.lowYRange]["E_lab"].max()
            )



    def getBassBarrierLab(self):
        A_p = self.isotopeA(self.projectile)
        Z_p = self.isotopeZ(self.projectile)

        A_t = self.isotopeA(self.target)
        Z_t = self.isotopeZ(self.target)

        R_12 = 1.07 * (A_p ** (1 / 3) + A_t ** (1 / 3))

        Bass_cm = Z_p * Z_t * 1.44 / (R_12 + 2.7) - 2.9 * (
            A_p ** (1 / 3) * A_t ** (1 / 3) / (A_p ** (1 / 3) + A_t ** (1 / 3))
        )

        return Bass_cm * (A_p + A_t) / A_t
    
    def getData(self):
        return self.data_ifus10
    
    def plot(
        self,
        plot_diff=True,
        plot_exp_data=True,
        alpha_value=0.15,
        show_reaction_info=True,
        reaction_info_note="",
        show_bass_barrier=True,
        display_plot=True,
        show_secondary_axis=True
    ):
        if not show_bass_barrier:
            show_bass_barrier = self.show_bass_barrier
        if self.reaction_info_note:
            reaction_info_note = self.reaction_info_note
        fig, ax = plt.subplots(figsize=(12, 5.5))

        sns.lineplot(
            x="E_lab",
            y="CS",
            hue="isotope",
            data=self.data_ifus10[
                self.data_ifus10["isotope"].isin(self.channels_to_plot)
            ],
            palette=colors,
        )
        sns.lineplot(
            x="E_lab",
            y="CS",
            hue="isotope",
            data=self.data_ifus0[
                self.data_ifus0["isotope"].isin(self.channels_to_plot)
            ],
            legend=None,
            linestyle="--",
            palette=colors,
        )
        for text in plt.legend(
            loc="upper right",
            framealpha=1,
            title=r"$^{"
            + str(self.isotopeA(self.cn))
            + "}$"
            + self.isotopeElement(self.cn)
            + r"$^*$",
            title_fontsize=18,
            fontsize=16,
        ).get_texts():
            label = text.get_text()
            text.set_text(
                "$^{" + str(self.isotopeA(label)) + "}$" + self.isotopeElement(label)
            )

        plt.semilogy()
        
        plt.ylim(self.lowYRange, self.highYRange)
        plt.xlim(self.lowXRange, self.highXRange)

        if show_secondary_axis:
            E_lab = self.data_ifus10['E_lab']
            E_exc = self.data_ifus10['E*/MeV']
            
            def forward(x):
                return np.interp(x, E_lab, E_exc)
            
            def inverse(x):
                return np.interp(x, E_exc, E_lab)
            
            secondary_axis = ax.secondary_xaxis('top', functions=(forward, inverse))
            
            secondary_axis.set_xlabel(r'$E^*$ [MeV]', fontsize=label_fontsize, labelpad=labelpad)
            
            exc_array = np.array(self.data_ifus10[self.data_ifus10['E_lab'] > self.lowXRange][self.data_ifus10['E_lab'] < self.highXRange]['E*/MeV'])
            
            secondary_ax_bin_separator = 5
            
            secondary_ax_bin_number = (max(exc_array) - (min(exc_array)//5+1)*5)/5
            
            if secondary_ax_bin_number >= 9:
                secondary_ax_bin_separator = 10
            elif secondary_ax_bin_number >= 20:
                secondary_ax_bin_separator = 20
            
            secondary_ax_ticks = np.array([int(i) for i in np.arange((min(exc_array)//secondary_ax_bin_separator+1)*secondary_ax_bin_separator, max(exc_array)+1, secondary_ax_bin_separator)])
            
            secondary_axis.set_xticks(secondary_ax_ticks)
            secondary_axis.set_xticklabels(secondary_ax_ticks)
            
        if self.unit == "ub":
            plt.ylabel("$\sigma$ [$\mu$b]", fontsize=label_fontsize)
        else:
            plt.ylabel(f"$\sigma$ [{self.unit}]", fontsize=label_fontsize)

        plt.xlabel("$E_{proj}$ [MeV]", fontsize=label_fontsize, labelpad=labelpad)

        if show_reaction_info:
            plt.text(
                0.02,
                0.97,
                r"$^{"
                + str(self.isotopeA(self.projectile))
                + "}$"
                + self.isotopeElement(self.projectile)
                + r" + $^{"
                + str(self.isotopeA(self.target))
                + "}$"
                + self.isotopeElement(self.target)
                + r" $\rightarrow$ $^{"
                + str(self.isotopeA(self.cn))
                + "}$"
                + self.isotopeElement(self.cn)
                + r"$^*$, BF = "
                + str(self.barfac)
                + str("" if self.bf_diff == 0 else (r" $\pm$ " + str(self.bf_diff))),
                ha="left",
                va="top",
                transform=ax.transAxes,
            )

            if reaction_info_note:
                plt.title(reaction_info_note, fontsize=12)

        if plot_diff and self.plot_diff and self.bf_diff > 0:
            self.__plotBfDiff(alpha_value)

        if plot_exp_data and self.plot_exp_data:
            self.__plotExpData()

        if (
            show_bass_barrier or self.show_bass_barrier
        ) and self.getBassBarrierLab() > (
            self.lowXRange + 5 / 100 * (self.highXRange - self.lowXRange)
        ):
            # plt.plot((self.getBassBarrierLab(), self.getBassBarrierLab()), (self.lowYRange, 10**(log10(self.lowYRange)+0.1*(log10(self.highYRange) - log10(self.lowYRange)))), color='black', linewidth=2)
            plt.annotate(
                r"$B_{Bass}$",
                xy=(self.getBassBarrierLab(), self.lowYRange),
                xytext=(
                    self.getBassBarrierLab(),
                    10
                    ** (
                        log10(self.lowYRange)
                        + 0.15 * (log10(self.highYRange) - log10(self.lowYRange))
                    ),
                ),
                ha="center",
                fontsize=label_fontsize,
                arrowprops=dict(arrowstyle="->", color="black", linewidth=2),
            )
            # plt.text(self.getBassBarrierLab(), 10**(log10(self.lowYRange)+0.15*(log10(self.highYRange) - log10(self.lowYRange))), r"$B_{Bass}$", horizontalalignment='center', verticalalignment='center', fontsize=label_fontsize)

        if display_plot:
            plt.show()
        return self

    def __plotBfDiff(self, alpha_value):
        try:
            for index, channel in enumerate(self.channels_to_plot):
                plt.fill_between(
                    self.data_ifus0[self.data_ifus0["isotope"] == channel]["E_lab"],
                    self.data_ifus0[self.data_ifus0["isotope"] == channel]["CS"],
                    self.data_ifus0_bf_low[
                        self.data_ifus0_bf_low["isotope"] == channel
                    ]["CS"],
                    color=colors[index],
                    alpha=alpha_value,
                )

                plt.fill_between(
                    self.data_ifus0[self.data_ifus0["isotope"] == channel]["E_lab"],
                    self.data_ifus0[self.data_ifus0["isotope"] == channel]["CS"],
                    self.data_ifus0_bf_high[
                        self.data_ifus0_bf_high["isotope"] == channel
                    ]["CS"],
                    color=colors[index],
                    alpha=alpha_value,
                )

                plt.fill_between(
                    self.data_ifus10[self.data_ifus10["isotope"] == channel]["E_lab"],
                    self.data_ifus10[self.data_ifus10["isotope"] == channel]["CS"],
                    self.data_ifus10_bf_low[
                        self.data_ifus10_bf_low["isotope"] == channel
                    ]["CS"],
                    color=colors[index],
                    alpha=alpha_value,
                )

                plt.fill_between(
                    self.data_ifus10[self.data_ifus10["isotope"] == channel]["E_lab"],
                    self.data_ifus10[self.data_ifus10["isotope"] == channel]["CS"],
                    self.data_ifus10_bf_high[
                        self.data_ifus10_bf_high["isotope"] == channel
                    ]["CS"],
                    color=colors[index],
                    alpha=alpha_value,
                )
        except:
            pass

    def __processExpData(self):
        self.__checkIsotopesInData()
        self.exp_data.scaleExpData(self.unit)

    def __checkIsotopesInData(self):
        exp_data_isotopes = set(self.exp_data.cs_data.keys())
        isotopes_to_show = set(self.channels_to_plot)

        if not exp_data_isotopes.issubset(isotopes_to_show):
            raise ValueError(
                "Experimental data contain isotopes which are not in the selected HIVAPs"
            )

    def __plotExpData(self):
        try:
            for isotope in self.exp_data.getExpEvapResidues():
                if self.plot_maxCS_data:
                    plt.axline(
                        (self.lowXRange, self.exp_data.getMaxChannelCS(isotope)),
                        (self.highXRange, self.exp_data.getMaxChannelCS(isotope)),
                        color=colors[self.channels_to_plot.index(isotope)],
                        ls=":",
                        linewidth=3,
                    )
                else:
                    for exp_data in self.exp_data.getEvapChannelExpData(isotope):
                        (_, caps, _) = plt.errorbar(
                            exp_data[0],
                            exp_data[1],
                            yerr=[[exp_data[2]], [exp_data[3]]],
                            marker=markers[self.channels_to_plot.index(isotope)],
                            color=colors[self.channels_to_plot.index(isotope)],
                            ls="none",
                            markersize=7,
                            capsize=5,
                        )
                        for cap in caps:
                            cap.set_markeredgewidth(1)
        except:
            print(
                f"No exp data entered for reaction {self.projectile} + {self.target} -> {self.cn}*"
            )

    def saveFig(self, save_note="", extension="pdf", relative_path=None):
        if self.save_note:
            save_note = self.save_note

        extension_types = ["pdf", "svg", "png"]
        if extension not in extension_types:
            raise ValueError(
                "Invalid extension type, please choose one from: %s" % extension_types
            )
        
        if relative_path == None:
            relative_path = "figs" + str("" if len(self.barrier_model) == 0 else "_" + self.barrier_model)
        else:
            relative_path = relative_path + str("" if len(self.barrier_model) == 0 else "_" + self.barrier_model)
            
        if not os.path.exists(relative_path):
            os.makedirs(relative_path)

        plt.savefig(
            relative_path
            + "/"
            + self.projectile
            + "_"
            + self.target
            + "_"
            + self.cn
            + "_"
            + self.evap_channel
            + "_barfac"
            + f"{self.barfac:.2f}"
            + "_sigr"
            + str(self.sigr)
            + str("" if len(save_note) == 0 else "_")
            + save_note
            + "."
            + extension,
            bbox_inches="tight",
        )

In [194]:
reactions = [

# RADONS

    Reaction('52Cr', '144Sm', '196Rn', barfac=0.47, sigr=2.5, bf_diff=0.02, channels_to_plot=['193Rn', '194Rn'], unit='nb',
exp_data=ExpData([232, 248], 
                 cs_data=
                 {
                 '193Rn': [np.nan, 50E-9],
                 '194Rn': [50e-9, np.nan],
                 },
                 error_data=
                 {
                 '193Rn': [[np.nan, 20e-9]],
                 '194Rn':[[50e-9, np.nan]],
                 })
),
    Reaction('56Fe', '142Nd', '198Rn', barfac=0.57, sigr=2.5, bf_diff=0.02, channels_to_plot=['195Rn', '196Rn'], unit='nb', plot_maxCS_data=True,
exp_data=ExpData([250], 
                 cs_data=
                 {
                 '195Rn': [0.00000204],
                 '196Rn': [0.00000196],
                 })
),
    Reaction('52Cr', '147Sm', '199Rn', barfac=0.58, sigr=3.0, bf_diff=0.02, channels_to_plot=['195Rn', '196Rn', '197Rn'], unit='nb',
exp_data=ExpData([221.8,224.9,232.6,235.8,252.1,260.5], 
                 cs_data=
                 {'197Rn': [5.95E-06,4.07E-06,6.53E-07,1.21E-06,2.15E-07,np.nan],
                 '196Rn': [np.nan,2.28E-07,4.45E-07,1.33E-06,9.21E-08,9.87E-08], 
                 '195Rn': [np.nan,np.nan,np.nan,np.nan,3.80E-07,2.15E-07]},
                 error_data=
                 {'197Rn': [[1.14E-06,1.21E-06,2.15E-07,1.98E-07,7.39E-08,np.nan], [1.14E-06,1.58E-06,3.22E-07,1.98E-07,1.01E-07,np.nan]],
                 '196Rn': [[np.nan,1.88E-07,1.48E-07,1.60E-07,3.76E-08,8.16E-08], [np.nan,5.24E-07,1.48E-07,1.60E-07,3.76E-08,2.27E-07]],
                 '195Rn': [[np.nan,np.nan,np.nan,np.nan,1.18E-07,1.78E-07],[np.nan,np.nan,np.nan,np.nan,1.24E-07,4.94E-07]]})
),
    Reaction('82Kr', '118Sn', '200Rn', barfac=0.61, sigr=2.5, bf_diff=0.02, channels_to_plot=['197Rn'], unit='nb',
exp_data=ExpData([362], 
                 cs_data=
                 {
                 '197Rn': [13e-6],
                 },
                 error_data=
                 {
                 '197Rn': [[3e-6]],
                 })
),
    Reaction('52Cr', '149Sm', '201Rn', barfac=0.63, sigr=3.0, bf_diff=0.02, channels_to_plot=['196Rn', '197Rn', '198Rn', '199Rn'], unit='nb',
exp_data=ExpData([218.9,222.1,229.4,233.6,236.2,244.5,251.3], 
                 cs_data=
                 {'196Rn': [np.nan,np.nan,np.nan,np.nan,np.nan,8.18E-08,1.70E-07], 
                 '197Rn': [np.nan,np.nan,1.38E-06,7.04E-06,1.55E-05,1.41E-05,6.16E-06],
                 '198Rn': [2.85E-05,4.09E-05,5.52E-05,4.73E-05,8.58E-06,2.12E-06,1.21E-06],
                 '199Rn': [3.64E-05,3.03E-05,1.76E-05,1.17E-05,np.nan,np.nan,np.nan]
                 },
                 error_data=
                 {'196Rn': [[np.nan,np.nan,np.nan,np.nan,np.nan,6.76E-08,1.10E-07], [np.nan,np.nan,np.nan,np.nan,np.nan,1.88E-07,2.24E-07]],
                 '197Rn': [[np.nan,np.nan,8.38E-08,1.25E-06,1.47E-06,1.12E-06,7.54E-07], [np.nan,np.nan,8.38E-08,1.25E-06,1.47E-06,1.12E-06,7.54E-07]],
                 '198Rn': [[1.01E-06,2.32E-06,5.78E-07,3.52E-06,1.19E-06,4.74E-07,3.66E-07], [1.01E-06,2.32E-06,5.78E-07,3.52E-06,1.19E-06,4.74E-07,3.66E-07]],
                 '199Rn':[[2.04E-06,3.37E-06,5.57E-07,2.62E-06,np.nan,np.nan,np.nan], [2.04E-06,3.37E-06,5.57E-07,2.62E-06,np.nan,np.nan,np.nan]]
                 })
),
    Reaction('52Cr', '150Sm', '202Rn', barfac=0.66, sigr=3.0, bf_diff=0.02, channels_to_plot=['198Rn', '199Rn', '200Rn'], unit='nb',
exp_data=ExpData([220.4,228.3,236.1], 
                 cs_data=
                 {
                 '198Rn': [9.77E-07,6.11E-06,9.11E-05],
                 '199Rn': [4.25E-04,6.40E-04,1.74E-04],
                 '200Rn': [1.26E-04,1.15E-04,1.68E-04]
                 },
                 error_data=
                 {
                 '198Rn': [[5.64E-07,3.94E-07,6.29E-06], [8.89E-07,3.94E-07,6.29E-06]],
                 '199Rn':[[2.40E-05,7.98E-06,1.73E-05], [2.40E-05,7.98E-06,1.73E-05]],
                 '200Rn':[[1.62E-05,4.33E-06,2.16E-05], [1.62E-05,4.33E-06,2.16E-05]]
                 })
),
    Reaction('36Ar', '166Er', '202Rn', barfac=0.70, sigr=3.2, bf_diff=0.02, channels_to_plot=['198Rn'], unit='nb',
exp_data=ExpData([175], 
                 cs_data=
                 {
                 '198Rn': [180e-6],
                 })
),
    Reaction('35Cl', '169Tm', '204Rn', barfac=0.71, sigr=3.3, bf_diff=0.02, channels_to_plot=['197Rn', '198Rn'], unit='nb',
exp_data=ExpData([196, 208.25], 
                 cs_data=
                 {
                 '197Rn': [np.nan,0.000022],
                 '198Rn': [0.00006,np.nan]
                 })
),
    Reaction('28Si', '176Hf', '204Rn', barfac=0.67, sigr=3.2, bf_diff=0.02, channels_to_plot=['200Rn'], unit='ub',
exp_data=ExpData([142], 
                 cs_data=
                 {
                 '200Rn': [5e-3],
                 })
),
    Reaction('45Sc', '159Tb', '204Rn', barfac=0.67, sigr=3.2, bf_diff=0.02, channels_to_plot=['199Rn', '200Rn'], unit='ub', evap_channel='xn', save_note='Werke',
             exp_data=ExpData(
                 E_lab=[195,195.8,199.4,201.2,203.9,205.8,213.3,217,221.4],
                 cs_data={
                     '199Rn': [np.nan,np.nan,np.nan,np.nan,0.0005,np.nan,np.nan,np.nan,np.nan],
                     '200Rn': [np.nan,np.nan,0.0024,np.nan,0.0008,np.nan,0.002,np.nan,np.nan]
                 },
                 error_data={
                     '199Rn': [[np.nan,np.nan,np.nan,np.nan,0.0003,np.nan,np.nan,np.nan,np.nan], [np.nan,np.nan,np.nan,np.nan,0.0004,np.nan,np.nan,np.nan,np.nan]],
                     '200Rn': [[np.nan,np.nan,0.0014,np.nan,0.0004,np.nan,0.0009,np.nan,np.nan], [np.nan,np.nan,0.0023,np.nan,0.0006,np.nan,0.0013,np.nan,np.nan]]
                 }
             )
    
),
    Reaction('45Sc', '159Tb', '204Rn', barfac=0.67, sigr=3.2, bf_diff=0.02, channels_to_plot=['198At', '199At', '200At', '201At'], unit='ub', evap_channel='pxn', save_note='Werke',
             exp_data=ExpData(
                 E_lab=[195,195.8,199.4,201.2,203.9,205.8,213.3,217,221.4],
                 cs_data={
                     '198At': [np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,0.015],
                     '199At': [np.nan,np.nan,0.0031,0.003,0.0095,0.017,np.nan,0.029,0.011],
                     '200At': [np.nan,0.04,0.039,0.026,0.052,0.054,0.031,0.019,np.nan],
                     '201At': [np.nan,np.nan,np.nan,np.nan,0.0098,0.0074,0.0081,np.nan,np.nan]
                 },
                 error_data={
                     '198At': [[np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,0.005],[np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,0.007]],
                     '199At': [[np.nan,np.nan,0.0016,0.0016,0.0014,0.003,np.nan,0.004,0.004],[np.nan,np.nan,0.0025,0.0025,0.0014,0.003,np.nan,0.004,0.006]],
                     '200At': [[np.nan,0.014,0.01,0.01,0.005,0.008,0.006,0.006,np.nan],[np.nan,0.014,0.01,0.01,0.005,0.008,0.006,0.006,np.nan]],
                     '201At': [[np.nan,np.nan,np.nan,np.nan,0.0023,0.0025,0.0026,np.nan,np.nan],[np.nan,np.nan,np.nan,np.nan,0.0023,0.0033,0.0034,np.nan,np.nan]]
                 }
             )
    
),
    Reaction('44Ca', '162Dy', '206Rn', barfac=0.68, sigr=3.2, bf_diff=0.02, channels_to_plot=['201Rn', '202Rn', '203Rn'], unit='ub',
exp_data=ExpData([183.7,188.9,195.4,199.5,207.1], 
                 cs_data=
                 {
                 '203Rn': [0.071,0.059,0.031,np.nan,np.nan],
                 '202Rn': [0.05,0.13,0.076,0.055,0.014],
                 '201Rn': [np.nan,np.nan,0.024,0.02,0.027]
                 },
                 error_data=
                 {
                 '203Rn': [[0.014,0.015,0.009,np.nan,np.nan], [0.017,0.019,0.011,np.nan,np.nan]],
                 '202Rn':[[0.011,0.02,0.014,0.01,0.005], [0.013,0.02,0.014,0.012,0.007]],
                 '201Rn':[[np.nan,np.nan,0.006,0.005,0.006], [np.nan,np.nan,0.007,0.007,0.007]]
                 })
),
    Reaction('48Ca', '162Dy', '210Rn', barfac=0.68, sigr=3.2, bf_diff=0.02, channels_to_plot=['204Rn', '207Rn'], unit='mb',
exp_data=ExpData([181.5,190.2,197.9,204.9,210.4], 
                 cs_data=
                 {
                 '207Rn': [1.6,1.7,1.3,0.5,np.nan],
                 '204Rn': [np.nan,np.nan,np.nan,0.3,1.1],
                 },
                 error_data=
                 {
                 '207Rn': [[0.4,0.9,0.3,0.2,np.nan]],
                 '204Rn':[[np.nan,np.nan,np.nan,0.1,0.2]],
                 })
),
    Reaction('22Ne', '190Os', '212Rn', barfac=0.79, sigr=3.0, bf_diff=0.02, channels_to_plot=['200Rn', '201Rn', '202Rn', '203Rn', '204Rn', '205Rn'], unit='ub', plot_maxCS_data=True, 
exp_data=ExpData([150], 
                 cs_data={
                     '205Rn': [27.7], 
                     '204Rn': [7.7], 
                     '203Rn': [3], 
                     '202Rn': [0.52], 
                     '201Rn': [0.058], 
                     '200Rn': [0.0036]
                 })),
    
    
#ASTATINES
    
    Reaction('90Zr', '103Rh', '193At', 0.41, 3.0, bf_diff=0.02, channels_to_plot=['190At'], unit='nb', exp_data=ExpData(
        E_lab=[410],
        cs_data={
        '190At': [0.13*1e-6]
        },
        error_data={
            '190At': [[0.065*1e-6]]
        }
    )),
    Reaction('51V', '144Sm', '195At', 0.60, 2.5, bf_diff=0.02, channels_to_plot=['192At'], unit='ub', save_note='SHIP', exp_data=ExpData(
        E_lab=[230], cs_data={'192At': [0.00004]}, error_data={'192At': [[0.00001]]}
    )),
    Reaction('54Fe', '141Pr', '195At', 0.55, 2.5, bf_diff=0.02, channels_to_plot=['191At'], unit='ub', save_note='RITU', exp_data=ExpData(
        E_lab=[260], cs_data={'191At': [0.0000003]}
    )),
    Reaction('56Fe', '141Pr', '197At', 0.64, 2.5, bf_diff=0.02, channels_to_plot=['193At', '194At', '195At'], unit='ub', save_note='SHIP', 
            exp_data=ExpData(
                E_lab=[229.4,235,243.4,251.8,265.8],
                cs_data={
                    '195At': [0.00013,0.00026,0.00013,0.000038,0.0000027],
                    '194At': [np.nan,0.000019,0.00018,0.0008,0.00018]
                },
                error_data={
                    '195At': [[0.00005,0.00008,0.00003,0.00001,0.0000013]],
                    '194At': [[np.nan,0.000009,0.00005,0.00024,0.00005]]
                }
            )),
    Reaction('56Fe', '141Pr', '197At', 0.64, 2.5, bf_diff=0.02, channels_to_plot=['193At', '194At', '195At'], unit='ub', save_note='RITU',
            exp_data=ExpData(
                E_lab=[230,248,266],
                cs_data={
                    '195At': [0.0009,np.nan,np.nan],
                    '194At': [np.nan,0.0015,np.nan],
                    '193At': [np.nan,np.nan,0.00004]
                }
            )),
    Reaction('56Fe', '141Pr', '197At', 0.64, 2.5, bf_diff=0.02, channels_to_plot=['193At', '194At', '195At'], unit='ub', save_note='both_exp',
            exp_data=ExpData(
                E_lab=[229.4,230,235,243.4,248,251.8,265.8,266],
                cs_data={
                    '195At': [0.00013,0.0009,0.00026,0.00013,np.nan,0.000038,0.0000027,np.nan],
                    '194At': [np.nan,np.nan,0.000019,0.00018,0.0015,0.0008,0.00018,np.nan],
                    '193At': [np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,0.00004],
                },
                error_data={
                    '195At': [[0.00005,np.nan,0.00008,0.00003,np.nan,0.00001,0.0000013,np.nan]],
                    '194At': [[np.nan,np.nan,0.000009,0.00005,np.nan,0.00024,0.00005,np.nan]],
                    '193At': [[np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan]],
                }
            )),
    Reaction('51V', '147Sm', '198At', 0.62, 3.0, bf_diff=0.02, channels_to_plot=['194At', '195At', '196At'], unit='ub',
            exp_data=ExpData(
            E_lab=[224],
                cs_data={
                    '194At': [50e-6],
                    '195At': [1200e-6],
                    '196At': [230e-6]
                }
            )),
    Reaction('40Ca', '159Tb', '199At', 0.72, 3.2, bf_diff=0.02, channels_to_plot=['194Po', '195Po', '196Po'], unit='ub', evap_channel='pxn',
            exp_data=ExpData(
                E_lab=[184,190,194,201,206],
                cs_data={
                    '196Po': [0.023,0.019,0.014,np.nan,0.0025],
                    '195Po': [0.041,0.11,0.102,0.04,0.03],
                    '194Po': [0.0005,0.004,0.015,0.042,0.089]
                }
            )),
    
    Reaction('45Sc', '156Gd', '201At', 0.61, 3.2, bf_diff=0.02, channels_to_plot=['197At', '198At'], unit='ub', save_note='Werke', 
            exp_data=ExpData(
                E_lab=[182.6,191.5,197.9,203.3,208.7],
                cs_data={
                    '198At': [0.0058,0.0031,np.nan,np.nan,np.nan],
                    '197At': [0.0012,0.0034,0.0057,np.nan,np.nan]
                },
                error_data={
                    '198At': [[0.0017,0.0013,np.nan,np.nan,np.nan], [0.0017,0.0019,np.nan,np.nan,np.nan]],
                    '197At': [[0.0006,0.0013,0.0021,np.nan,np.nan], [0.0009,0.0018,0.0021,np.nan,np.nan]]
                }
            )),
    Reaction('45Sc', '156Gd', '201At', 0.61, 3.2, bf_diff=0.02, channels_to_plot=['196Po', '197Po', '198Po'], unit='ub', evap_channel='pxn', save_note='Werke',
            exp_data=ExpData(
                E_lab=[182.6,191.5,197.9,203.3,208.7],
                cs_data={
                    '198Po': [0.0036,np.nan,np.nan,np.nan,np.nan],
                    '197Po': [0.0014,0.026,0.032,0.024,0.011],
                    '196Po': [np.nan,np.nan,np.nan,0.013,0.034]
                },
                error_data={
                    '198Po': [[0.0013,np.nan,np.nan,np.nan,np.nan],[0.0019,np.nan,np.nan,np.nan,np.nan]],
                    '197Po': [[0.0007,0.005,0.005,0.005,0.003],[0.0011,0.005,0.005,0.005,0.003]],
                    '196Po': [[np.nan,np.nan,np.nan,0.003,0.005],[np.nan,np.nan,np.nan,0.003,0.005]]
                }
            )),
    
    Reaction('45Sc', '157Gd', '202At', 0.62, 3.2, bf_diff=0.02, channels_to_plot=['197At', '198At', '199At'], unit='ub', save_note='Werke',
            exp_data=ExpData(
                E_lab=[185.3,190.9,194.2,198.2,200.5,205.9,211.3],
                cs_data={
                    '199At': [0.0098,0.0067,0.014,0.0029,0.0037,np.nan,np.nan],
                    '198At': [np.nan,0.025,0.022,0.02,0.014,0.0033,np.nan],
                    '197At': [np.nan,np.nan,np.nan,np.nan,0.0035,np.nan,np.nan]
                },
                error_data={
                    '199At': [[0.0034,0.0023,0.004,0.0015,0.0016,np.nan,np.nan],[0.0046,0.0031,0.004,0.0024,0.0023,np.nan,np.nan]],
                    '198At': [[np.nan,0.005,0.005,0.005,0.004,0.0018,np.nan],[np.nan,0.005,0.005,0.005,0.004,0.0029,np.nan]],
                    '197At': [[np.nan,np.nan,np.nan,np.nan,0.0015,np.nan,np.nan],[np.nan,np.nan,np.nan,np.nan,0.0021,np.nan,np.nan]]
                }
            )),
    Reaction('45Sc', '157Gd', '202At', 0.62, 3.2, bf_diff=0.02, channels_to_plot=['196Po', '197Po', '198Po', '199Po'], unit='ub', evap_channel='pxn', save_note='Werke',
            exp_data=ExpData(
                E_lab=[185.3,190.9,194.2,198.2,200.5,205.9,211.3],
                cs_data={
                    '199Po': [np.nan,np.nan,0.036,np.nan,0.049,np.nan,np.nan],
                    '198Po': [np.nan,0.018,0.008,0.039,0.042,0.015,0.01],
                    '197Po': [np.nan,0.0059,np.nan,0.018,0.028,0.029,0.042],
                    '196Po': [np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,0.0069]
                },
                error_data={
                    '199Po': [[np.nan,np.nan,0.015,np.nan,0.018,np.nan,np.nan],[np.nan,np.nan,0.021,np.nan,0.025,np.nan,np.nan]],
                    '198Po': [[np.nan,0.006,0.0029,0.008,0.008,0.006,0.004],[np.nan,0.006,0.0039,0.008,0.008,0.006,0.006]],
                    '197Po': [[np.nan,0.0023,np.nan,0.005,0.006,0.007,0.009],[np.nan,0.0032,np.nan,0.005,0.006,0.007,0.009]],
                    '196Po': [[np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,0.0028],[np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,0.004]]
                }
            )),
    
    Reaction('45Sc', '158Gd', '203At', 0.60, 3.2, bf_diff=0.02, channels_to_plot=['198At', '199At', '200At'], unit='ub', save_note='Werke',
            exp_data=ExpData(
                E_lab=[180.4,185.9,189.1,195.3,201.3],
                cs_data={
                    '200At': [0.015,0.0087,0.0083,np.nan,np.nan],
                    '199At': [0.0071,0.034,0.039,0.033,0.017],
                    '198At': [np.nan,np.nan,np.nan,0.003,0.011],
                },
                error_data={
                    '200At': [[0.008,0.0042,0.0037,np.nan,np.nan],[0.013,0.0062,0.0052,np.nan,np.nan]],
                    '199At': [[0.0043,0.008,0.007,0.005,0.004],[0.0068,0.008,0.007,0.005,0.004]],
                    '198At': [[np.nan,np.nan,np.nan,0.0013,0.004],[np.nan,np.nan,np.nan,0.0019,0.004]],
                }
            )),
    Reaction('45Sc', '158Gd', '203At', 0.60, 3.2, bf_diff=0.02, channels_to_plot=['198Po', '199Po', '200Po'], unit='ub', evap_channel='pxn', save_note='Werke',
            exp_data=ExpData(
                E_lab=[180.4,185.9,189.1,195.3,201.3],
                cs_data={
                    '200Po': [0.069,np.nan,np.nan,np.nan,np.nan],
                    '199Po': [np.nan,np.nan,0.026,0.017,0.088],
                    '198Po': [np.nan,np.nan,np.nan,0.0035,0.023],
                },
                error_data={
                    '200Po': [[0.038,np.nan,np.nan,np.nan,np.nan],[0.059,np.nan,np.nan,np.nan,np.nan]],
                    '199Po': [[np.nan,np.nan,0.015,0.009,0.033],[np.nan,np.nan,0.024,0.015,0.033]],
                    '198Po': [[np.nan,np.nan,np.nan,0.0017,0.007],[np.nan,np.nan,np.nan,0.0026,0.007]],
                }
            )),
    
    Reaction('44Ca', '159Tb', '203At', 0.64, 3.2, bf_diff=0.02, channels_to_plot=['198At', '199At', '200At'], unit='ub', save_note='Werke',
            exp_data=ExpData(
                E_lab=[184.7,189.8,196.3,200.5,208],
                cs_data={
                    '200At': [0.14,0.09,0.023,np.nan,np.nan],
                    '199At': [0.13,0.23,0.12,0.041,np.nan],
                    '198At': [np.nan,0.018,0.034,0.048,0.033],
                },
                error_data={
                    '200At': [[0.03,0.021,0.008,np.nan,np.nan],[0.04,0.025,0.011,np.nan,np.nan]],
                    '199At': [[0.02,0.02,0.01,0.008,np.nan],[0.02,0.02,0.01,0.01,np.nan]],
                    '198At': [[np.nan,0.006,0.007,0.009,0.007],[np.nan,0.008,0.008,0.011,0.009]],
                }
            )),
    Reaction('44Ca', '159Tb', '203At', 0.64, 3.2, bf_diff=0.02, channels_to_plot=['197Po', '198Po', '199Po'], unit='ub', evap_channel='pxn', save_note='Werke',
            exp_data=ExpData(
                E_lab=[184.7,189.8,196.3,200.5,208],
                cs_data={
                    '199Po': [np.nan,0.17,0.22,0.2,0.16],
                    '198Po': [np.nan,np.nan,0.033,0.051,0.05],
                    '197Po': [np.nan,np.nan,np.nan,np.nan,0.038],
                },
                error_data={
                    '199Po': [[np.nan,0.06,0.05,0.06,0.05],[np.nan,0.07,0.06,0.08,0.07]],
                    '198Po': [[np.nan,np.nan,0.008,0.012,0.011],[np.nan,np.nan,0.01,0.015,0.014]],
                    '197Po': [[np.nan,np.nan,np.nan,np.nan,0.009],[np.nan,np.nan,np.nan,np.nan,0.011]],
                }
            )),
    
    Reaction('45Sc', '160Gd', '205At', 0.63, 3.2, bf_diff=0.02, channels_to_plot=['199At', '200At', '201At', '202At'], unit='ub', save_note='Werke',
             exp_data=ExpData(
                E_lab=[181.2,186.6,189.9,194.3,202],
                cs_data={
                    '202At': [np.nan,0.041,np.nan,0.012,np.nan],
                    '201At': [0.1,0.15,0.12,0.14,0.028],
                    '200At': [np.nan,0.035,0.1,0.14,0.33],
                    '199At': [np.nan,np.nan,np.nan,0.0024,0.041]
                },
                error_data={
                    '202At': [[np.nan,0.014,np.nan,0.004,np.nan],[np.nan,0.014,np.nan,0.006,np.nan]],
                    '201At': [[0.02,0.02,0.02,0.01,0.009],[0.02,0.02,0.02,0.01,0.009]],
                    '200At': [[np.nan,0.01,0.02,0.01,0.03],[np.nan,0.01,0.02,0.01,0.03]],
                    '199At': [[np.nan,np.nan,np.nan,0.001,0.007],[np.nan,np.nan,np.nan,0.0014,0.007]]
                }
            )),
    Reaction('45Sc', '160Gd', '205At', 0.63, 3.2, bf_diff=0.02, channels_to_plot=['199Po', '200Po', '201Po'], unit='ub', save_note='Werke', evap_channel='pxn',
             exp_data=ExpData(
                E_lab=[181.2,186.6,189.9,194.3,202],
                cs_data={
                    '201Po': [np.nan,np.nan,0.15,np.nan,np.nan],
                    '200Po': [np.nan,np.nan,np.nan,0.038,np.nan],
                    '199Po': [np.nan,np.nan,np.nan,np.nan,0.055]
                },
                error_data={
                    '201Po': [[np.nan,np.nan,0.08,np.nan,np.nan],[np.nan,np.nan,0.12,np.nan,np.nan]],
                    '200Po': [[np.nan,np.nan,np.nan,0.011,np.nan],[np.nan,np.nan,np.nan,0.015,np.nan]],
                    '199Po': [[np.nan,np.nan,np.nan,np.nan,0.022],[np.nan,np.nan,np.nan,np.nan,0.03]]
                }
            )),    
    
    Reaction('24Mg', '181Ta', '205At', 0.74, 3.2, bf_diff=0.02, channels_to_plot=['197At', '198At', '199At', '200At'], unit='mb', save_note='Andreyev',
            exp_data=ExpData(
                E_lab=[118.8,128.8,135.8,137.6,141.8,151.8,162.8,167.9,172.8],
                cs_data={
                    '200At': [24.3,29.7,8.1,3.3,1.72,0.15,np.nan,np.nan,np.nan],
                    '199At': [0.027,1.98,4.4,3.8,2.9,0.54,0.035,0.044,0.012],
                    '198At': [np.nan,np.nan,0.06,0.293,0.65,0.85,0.27,0.21,0.077],
                    '197At': [np.nan,np.nan,np.nan,np.nan,np.nan,0.007,0.05,0.054,0.04],                    
                },
            )),
    Reaction('24Mg', '181Ta', '205At', 0.74, 3.2, bf_diff=0.02, channels_to_plot=['196Po', '197Po', '198Po', '199Po'], unit='mb', evap_channel='pxn', save_note='Andreyev',
            exp_data=ExpData(
                E_lab=[118.8,128.8,135.8,137.6,141.8,151.8,162.8,167.9,172.8],
                cs_data={
                    '199Po': [0.022,0.9,2.2,2.9,2.9,1.1,0.16,0.16,0.054],
                    '198Po': [np.nan,np.nan,0.12,0.33,0.74,1.54,1,1.1,0.45],
                    '197Po': [np.nan,np.nan,np.nan,np.nan,np.nan,0.181,1.1,1.2,1.35],
                    '196Po': [np.nan,np.nan,np.nan,np.nan,np.nan,0.025,0.058,0.055,0.25],
                },
            )),
    
    Reaction('40Ar', '165Ho', '205At', 0.75, 3.3, bf_diff=0.02, channels_to_plot=['199At', '200At', '201At', '202At'], unit='mb', save_note='Vermeulen',
            exp_data=ExpData(
                E_lab=[164.9,170.6,178.1,184.3,186.3,192,199.2],
                cs_data={
                    '202At': [4.4,2.3,0.309,np.nan,np.nan,np.nan,np.nan],
                    '201At': [3.1,11.3,7.6,1.7,1.4,0.547,0.104],
                    '200At': [np.nan,0.356,3.6,9.8,9.1,8.2,0.965],
                    '199At': [np.nan,np.nan,7.31E-04,2.39E-01,3.39E-01,1.1,2],                    
                },
                error_data={
                    '202At': [[0.0748,0.069,0.0309,np.nan,np.nan,np.nan,np.nan]],
                    '201At': [[0.0496,0.113,0.1216,0.0867,0.0224,0.08752,0.003432]],
                    '200At': [[np.nan,0.016732,0.054,0.1666,0.0546,0.246,0.012545]],
                    '199At': [[np.nan,np.nan,4.24E-04,1.91E-02,8.81E-03,0.0605,0.012]],                
                }
            )),
    
    Reaction('40Ar', '165Ho', '205At', 0.74, 3.3, bf_diff=0.02, channels_to_plot=['196At', '197At', '198At', '199At', '200At'], unit='mb', save_note='Andreyev',
            exp_data=ExpData(
                E_lab=[175.6,180,182.4,185.5,189.2,194.8,198.5,199.8,202.9,207.2,211,219,224.7,231.5,237.1,241.4,249.5,256.3,263.8,271.2,279.3],
                cs_data={
                    '200At': [2.43,9.3,8.25,11.5,8.8,4.65,2.6,1.7,1.25,0.35,0.21,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan],
                    '199At': [np.nan,np.nan,0.1,0.46,0.85,1.17,1.93,1.62,1.95,0.8,0.32,0.1,0.02,0.028,np.nan,0.01,np.nan,np.nan,np.nan,np.nan,np.nan],
                    '198At': [np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,0.035,0.1,0.3,0.35,0.37,0.16,0.15,0.035,0.056,0.017,0.009,np.nan,np.nan,np.nan],
                    '197At': [np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,0.015,0.035,0.028,0.035,0.031,0.016,0.004,0.0015,0.0008,np.nan],
                    '196At': [np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,0.001,0.0014,0.003,0.0035,0.0017,0.001,np.nan],
                },
            )),
    Reaction('40Ar', '165Ho', '205At', 0.74, 3.3, bf_diff=0.02, channels_to_plot=['195Po', '196Po', '197Po', '198Po', '199Po'], unit='mb', evap_channel='pxn', save_note='Andreyev',
            exp_data=ExpData(
                E_lab=[175.6,180,182.4,185.5,189.2,194.8,198.5,199.8,202.9,207.2,211,219,224.7,231.5,237.1,241.4,249.5,256.3,263.8,271.2,279.3],
                cs_data={
                    '199Po': [np.nan,np.nan,0.1,0.46,0.32,0.55,0.65,0.71,1.34,0.91,0.6,0.25,0.25,0.07,0.06,0.031,0.015,np.nan,np.nan,np.nan,np.nan],
                    '198Po': [np.nan,np.nan,np.nan,np.nan,np.nan,0.09,0.075,0.105,0.25,0.62,0.75,1.05,0.712,0.475,0.26,0.256,0.081,0.024,np.nan,np.nan,np.nan],
                    '197Po': [np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,0.1,0.41,1,0.6,1,0.79,0.56,0.22,0.09,0.036,np.nan],
                    '196Po': [np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,0.015,0.03,0.084,0.15,0.18,0.345,0.33,0.23,0.14,0.056],
                    '195Po': [np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,0.019,0.054,0.09,0.12,0.072],
                },
            )),

    Reaction('40Ar', '165Ho', '205At', 0.68, 3.3, bf_diff=0.02, channels_to_plot=['199At'], unit='mb', save_note='Jakobsson',
            exp_data=ExpData(
                E_lab=[200],
                cs_data={
                    '199At': [0.7],                    
                },
            )),
    
    Reaction('40Ar', '165Ho', '205At', 0.74, 3.3, bf_diff=0.02, channels_to_plot=['198At', '199At', '200At'], unit='mb', save_note='Folden',
            exp_data=ExpData(
                E_lab=[180.3,188.4,196.5,202.6,208.6,213.3,218.5],
                cs_data={
                    '200At': [9.1,5,1.19,0.13,np.nan,np.nan,np.nan],
                    '199At': [0.2,1.21,2.29,1.54,0.73,0.28,0.1],                    
                    '198At': [np.nan,np.nan,np.nan,0.1,0.21,0.18,0.08],                    
                },
                error_data={
                    '200At': [[3.3,1.8,0.44,0.06,np.nan,np.nan,np.nan]],
                    '199At': [[0.12,0.43,0.8,0.54,0.26,0.1,0.04]],
                    '198At': [[np.nan,np.nan,np.nan,0.05,0.08,0.07,0.03]],
                }
            )),
    Reaction('40Ar', '165Ho', '205At', 0.74, 3.3, bf_diff=0.02, channels_to_plot=['197Po', '198Po', '199Po'], unit='mb', evap_channel='pxn', save_note='Folden',
            exp_data=ExpData(
                E_lab=[180.3,188.4,196.5,202.6,208.6,213.3,218.5],
                cs_data={
                    '199Po': [0.04,0.42,1.03,1.64,1.19,0.72,0.34],
                    '198Po': [np.nan,np.nan,np.nan,0.26,0.78,1.13,0.96],
                    '197Po': [np.nan,np.nan,np.nan,np.nan,0.02,0.06,0.17],                
                },
                error_data={
                    '199Po': [[0.03,0.17,0.38,0.59,0.43,0.26,0.13]],
                    '198Po': [[np.nan,np.nan,np.nan,0.1,0.28,0.4,0.34]],
                    '197Po': [[np.nan,np.nan,np.nan,np.nan,0.01,0.03,0.07]],              
                }
            )),
    
    Reaction('26Mg', '181Ta', '207At', 0.76, 3.2, bf_diff=0.02, channels_to_plot=['200At', '201At', '202At', '203At'], unit='mb', plot_maxCS_data=True, save_note='Yeremin',
            exp_data=ExpData(
            E_lab = [150],
                cs_data={
                    '203At': [14],
                    '202At': [40],
                    '201At': [20],
                    '200At': [8]
                }
            )),
    
    Reaction('48Ca', '159Tb', '207At', 0.78, 3.2, bf_diff=0.02, channels_to_plot=['201At', '202At', '203At', '204At'], unit='mb', save_note='Mayorov',
            exp_data=ExpData(
                E_lab=[185.1,190.8,193,193.5,197.8,198,201.3,203.1,204.1,209.4],
                cs_data={
                    '204At': [5.1,4.2,3.5,2.5,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan],
                    '203At': [3,6.1,10.9,12.6,12.5,10.7,7,5.8,3.1,2.2],
                    '202At': [np.nan,np.nan,1.5,1.3,1.6,2.7,16.7,17.5,23.3,21.1],
                    '201At': [np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,0.8],                    
                },
                error_data={
                    '204At': [[1.7,3,2,1.6,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan]],
                    '203At': [[0.6,1.4,1.8,1.9,2,1.6,1.1,1.9,0.6,2]],
                    '202At': [[np.nan,np.nan,0.5,0.5,0.7,0.6,4.5,7,7.1,7.8]],
                    '201At': [[np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,0.2]],
                }
            )),
]

Using CPS calculations
Using CPS calculations
Using CPS calculations
Using CPS calculations
Using CPS calculations
Using CPS calculations
Using CPS calculations
Using CPS calculations
Using CPS calculations
Using CPS calculations
Using CPS calculations
Using CPS calculations
Using CPS calculations
Using CPS calculations
Using CPS calculations
Using CPS calculations
Using CPS calculations
Using CPS calculations
Using CPS calculations
Using CPS calculations
Using CPS calculations
Using CPS calculations
Using CPS calculations
Using CPS calculations
Using CPS calculations
Using CPS calculations
Using CPS calculations
Using CPS calculations
Using CPS calculations
Using CPS calculations
Using CPS calculations
Using CPS calculations
Using CPS calculations
Using CPS calculations
Using CPS calculations
Using CPS calculations
Using CPS calculations
Using CPS calculations
Using CPS calculations
Using CPS calculations
Using CPS calculations
Using CPS calculations


In [196]:
start = time.time()
for reaction in reactions:
    reaction.plot(show_reaction_info=True, show_bass_barrier=True, display_plot=False)
    reaction.saveFig()
    
end = time.time()

print('Execution time:', end-start, 's')

Execution time: 66.42760419845581 s
